In [21]:
#ML expects time as rows You can’t easily calculate trends

In [22]:
import pandas as pd
import numpy as np

In [23]:
df = pd.read_csv("/content/Financials.csv")
df.head()

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Government,Canada,Carretera,None,"$1,618.50",$3.00,$20.00,"$32,370.00",$-,"$32,370.00","$16,185.00","$16,185.00",01/01/2014,1,January,2014
1,Government,Germany,Carretera,None,"$1,321.00",$3.00,$20.00,"$26,420.00",$-,"$26,420.00","$13,210.00","$13,210.00",01/01/2014,1,January,2014
2,Midmarket,France,Carretera,None,"$2,178.00",$3.00,$15.00,"$32,670.00",$-,"$32,670.00","$21,780.00","$10,890.00",01/06/2014,6,June,2014
3,Midmarket,Germany,Carretera,None,$888.00,$3.00,$15.00,"$13,320.00",$-,"$13,320.00","$8,880.00","$4,440.00",01/06/2014,6,June,2014
4,Midmarket,Mexico,Carretera,None,"$2,470.00",$3.00,$15.00,"$37,050.00",$-,"$37,050.00","$24,700.00","$12,350.00",01/06/2014,6,June,2014


In [24]:
df.columns

Index(['Segment', 'Country', ' Product ', ' Discount Band ', ' Units Sold ',
       ' Manufacturing Price ', ' Sale Price ', ' Gross Sales ', ' Discounts ',
       '  Sales ', ' COGS ', ' Profit ', 'Date', 'Month Number',
       ' Month Name ', 'Year'],
      dtype='object')

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Segment                700 non-null    object
 1   Country                700 non-null    object
 2    Product               700 non-null    object
 3    Discount Band         700 non-null    object
 4    Units Sold            700 non-null    object
 5    Manufacturing Price   700 non-null    object
 6    Sale Price            700 non-null    object
 7    Gross Sales           700 non-null    object
 8    Discounts             700 non-null    object
 9     Sales                700 non-null    object
 10   COGS                  700 non-null    object
 11   Profit                700 non-null    object
 12  Date                   700 non-null    object
 13  Month Number           700 non-null    int64 
 14   Month Name            700 non-null    object
 15  Year                   

In [26]:
df.columns = df.columns.str.lower().str.replace(" ", "_")

In [27]:
df.columns = (
    df.columns
    .str.lower()
    .str.strip()
    .str.replace(r"\s+", "_", regex=True)  # replace ANY whitespace with single _
    .str.strip("_")                        # remove leading/trailing underscores
)

In [28]:
df.columns

Index(['segment', 'country', 'product', 'discount_band', 'units_sold',
       'manufacturing_price', 'sale_price', 'gross_sales', 'discounts',
       'sales', 'cogs', 'profit', 'date', 'month_number', 'month_name',
       'year'],
      dtype='object')

In [29]:
num_cols = [
    "units_sold", "cogs", "manufacturing_price", "sale_price",
    "gross_sales", "discounts", "sales"
]

for col in num_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace("$", "")
        .str.replace(",", "")
        .str.replace("-", "0")
        .astype(float)
    )

In [30]:
#checing if gross sales formula is correctlt used, difference should be close to 0
(df["units_sold"] * df["sale_price"] - df["gross_sales"]).abs().describe()

,0
count,700.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


In [31]:
(df["gross_sales"] - df["discounts"] - df["sales"]).abs().describe()

,0
count,700.000000
mean,0.000086
std,0.000923
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,0.010000


In [32]:
(df["units_sold"] * df["manufacturing_price"] - df["cogs"]).abs().describe()

,0
count,700.000000
mean,166565.392857
std,207620.850833
min,0.000000
25%,2221.000000
50%,70369.000000
75%,294746.250000
max,931612.500000


In [33]:
df[num_cols].dtypes

,0
units_sold,float64
cogs,float64
manufacturing_price,float64
sale_price,float64
gross_sales,float64
discounts,float64
sales,float64


In [34]:
df[["cogs", "profit"]].dtypes

,0
cogs,float64
profit,object


In [35]:
df["profit_calc"] = df["sales"] - df["cogs"]

In [40]:
(df["profit_calc"] - df["profit"]).abs().describe()

,0
count,6.950000e+02
mean,2.236896e+03
std,9.457522e+03
min,0.000000e+00
25%,0.000000e+00
50%,0.000000e+00
75%,5.684342e-13
max,8.123500e+04


In [37]:
df[df["profit"].apply(lambda x: isinstance(x, str))]["profit"].unique()[:10]

array([' $16,185.00 ', ' $13,210.00 ', ' $10,890.00 ', ' $4,440.00 ',
       ' $12,350.00 ', ' $1,36,170.00 ', ' $4,605.00 ', ' $22,662.00 ',
       ' $18,990.00 ', ' $13,905.00 '], dtype=object)

In [38]:
df["profit"] = pd.to_numeric(
    df["profit"]
        .astype(str)
        .str.replace(r"[^0-9.\-]", "", regex=True),
    errors="coerce"
)

In [39]:
df["date"] = pd.to_datetime(df["date"])
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month

In [41]:
df["month_name"] = df["date"].dt.month_name()

In [42]:
df.to_csv("cleaned_sales_data.csv", index=False)

In [43]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [44]:
!ls /content/drive/MyDrive

'021,049,065-LW Poster.pdf'
 2.jpg
 3CF7E00E-ABAC-4AB0-8278-24880FFE0101.collage
 95ae7111-3ac0-4703-a41d-bdf0e9eb59b3.jpeg
'ACE- Presentation [S002] (1).pptx'
'ACE- Presentation [S002].pptx'
'ANNUAL DAY.jpg'
'Assignment 1_E053 .docx'
'Aws  (File responses)'
 aws.gform
'Aws .gform'
 Aws.gform
 Cancer_Data.xlsx
'Colab Notebooks'
'Copy of bd .gsheet'
'Copy of Metaverse survey.gform'
'D and f block Quick notes - canvas classes.gdoc'
 E051.pdf
 e053_exp1.docx
 e053_exp2.docx
 E053_exp3.docx
'E053_Sakshi Rohida_email (1).docx'
'E053_Sakshi Rohida_email (2).docx'
'E053_Sakshi Rohida_email.docx'
 E053_SakshiRohida_Exp10.pdf
'E053_Sakshi Rohida_Exp5.pdf'
 E053_SakshiRohida_Exp6.pdf
 E053_SakshiRohida_Exp7.pdf
 E053_SkashiRohida_CSBS_evs.pdf
 Eng-Core_2021-22.pdf
'Getting started.pdf'
'Google Earth'
'Guys, high fucking ambitions.gdoc'
 IETE_Main_Proposal.pdf
 IIIPME.gdoc
 IMG_0488.PNG
 IMG_0675.HEIC
 IMG_0676.HEIC
 IMG_0723.png
 IMG_0723.PNG
 IMG_0741.png
 IMG_1845.JPG
 IMG_2143.PNG
'MWA synops

In [45]:
import os

In [46]:
project_path = "/content/drive/MyDrive/finance_sales_project"

In [47]:
os.makedirs(project_path, exist_ok=True)

In [49]:
df.to_csv("/content/drive/MyDrive/finance_sales_project/cleaned_sales_data.csv", index=False)

In [50]:
!ls /content/drive/MyDrive/finance_sales_project

cleaned_sales_data.csv


In [51]:
!ls /content/drive/MyDrive

'021,049,065-LW Poster.pdf'
 2.jpg
 3CF7E00E-ABAC-4AB0-8278-24880FFE0101.collage
 95ae7111-3ac0-4703-a41d-bdf0e9eb59b3.jpeg
'ACE- Presentation [S002] (1).pptx'
'ACE- Presentation [S002].pptx'
'ANNUAL DAY.jpg'
'Assignment 1_E053 .docx'
'Aws  (File responses)'
 aws.gform
'Aws .gform'
 Aws.gform
 Cancer_Data.xlsx
'Colab Notebooks'
'Copy of bd .gsheet'
'Copy of Metaverse survey.gform'
'D and f block Quick notes - canvas classes.gdoc'
 E051.pdf
 e053_exp1.docx
 e053_exp2.docx
 E053_exp3.docx
'E053_Sakshi Rohida_email (1).docx'
'E053_Sakshi Rohida_email (2).docx'
'E053_Sakshi Rohida_email.docx'
 E053_SakshiRohida_Exp10.pdf
'E053_Sakshi Rohida_Exp5.pdf'
 E053_SakshiRohida_Exp6.pdf
 E053_SakshiRohida_Exp7.pdf
 E053_SkashiRohida_CSBS_evs.pdf
 Eng-Core_2021-22.pdf
 finance_sales_project
'Getting started.pdf'
'Google Earth'
'Guys, high fucking ambitions.gdoc'
 IETE_Main_Proposal.pdf
 IIIPME.gdoc
 IMG_0488.PNG
 IMG_0675.HEIC
 IMG_0676.HEIC
 IMG_0723.png
 IMG_0723.PNG
 IMG_0741.png
 IMG_1845.JPG
 I

In [52]:
!ls /content/drive/MyDrive/finance_sales_project

cleaned_sales_data.csv
